In [2]:
import json
import datetime
import numpy as np
from glob import glob
import os.path as osp
from easydict import EasyDict as edict
from collections import defaultdict

train_path = '/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/train'
action_output_path = '/home/zal/Devel/SyntheticProposals/data/umd_v3/sitting_down.json'

### Extract the annotations for specific action

In [3]:
def get_name(input_path):
    return osp.splitext(osp.basename(input_path))[0]

def get_bbox_from_traj(traj):
    bboxes = np.array([bbox for _, bbox in traj.items()])
    y1, x1, y2, x2 = [int(np.min(bboxes[:, 0])), 
                        int(np.min(bboxes[:, 1])), 
                        int(np.max(bboxes[:, 2])), 
                        int(np.max(bboxes[:, 3]))]
    w, h = x2-x1, y2-y1
    cx = x1 + w//2
    cy = y1 + h//2
    side = w if w > h else h
    hside = side//2
    return [cy-hside, cx-hside, cy+hside, cx+hside]

In [4]:
action = 'person_sitting_down'
action_json = dict()
index = 0
json_path_list = glob(osp.join(train_path, '*.json'))
for path_list in json_path_list:
    video_json = edict(json.load(open(path_list)))
    for key in video_json.keys():
        video_anno = video_json[key]
        duration = video_anno.end_frame - video_anno.start_frame
        if (video_anno.event_type == action) and (duration < 1000):
            action_json[index] = video_anno
            action_json[index].video_name = get_name(path_list)
            action_json[index].bbox = get_bbox_from_traj(video_anno.trajectory)
            action_json[index].duration = action_json[index].end_frame - action_json[index].start_frame
            del action_json[index]['objects']
            del action_json[index]['trajectory']
            index += 1

with open(action_output_path, 'w') as output_file:
    json.dump(action_json, output_file)

print(f'Total actions: {len(action_json)}')
# Test the output
print(action_json[1])

Total actions: 315
{'start_frame': 195, 'end_frame': 253, 'event_type': 'person_sitting_down', 'video_name': '2018-03-07.16-50-00.16-55-00.bus.G331', 'bbox': [1384, 170, 1738, 524], 'duration': 58}


### Extract Frames

In [5]:
import os
import imageio
from PIL import Image

video_dir ='/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/videos'
output_frames_dir = '/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/proposals/frames'

In [6]:
def extract_frames(video_path, bbox, start_frame, end_frame, output_dir):
    video_reader = imageio.get_reader(video_path)
    for frame_idx, frame_arr in enumerate(video_reader):
        if frame_idx > end_frame:
            break
        if (frame_idx >= start_frame) and (frame_idx <= end_frame):
            Image.fromarray(frame_arr).crop(bbox).save(osp.join(output_dir, f'{frame_idx}.png'))

In [ ]:
for action_id, annotation in action_json.items():
    # Normalize parameters
    video_name = annotation.video_name.replace('.', '_')
    anno = edict(annotation)
        
    video_path = osp.join(video_dir, f'{video_name}.avi')
    output_dir = osp.join(output_frames_dir, f'{video_name}_{action_id}')
    
    try:
        os.makedirs(output_dir, exist_ok=False)
        print(datetime.datetime.now().time(), anno.start_frame, anno.duration, f'{video_name}_{action_id}')
    except:
        continue
    extract_frames(video_path, anno.bbox, anno.start_frame, anno.end_frame, output_dir)

### Extract empty seats

In [24]:
empty_frames_dir = '/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/proposals/empty_frames'

prop_11 = edict(video_path=osp.join(video_dir, '2018-03-07_16-55-00_17-00-00_bus_G331.avi'),
              bbox=[448, 160, 764, 476],
              start_frame=0,
              end_frame=56,
              output_dir=osp.join(empty_frames_dir, '2018-03-07_16-55-00_17-00-00_bus_G331_11'))

prop_11_alt = edict(video_path=osp.join(video_dir, '2018-03-07_16-55-00_17-00-00_bus_G331.avi'),
              bbox=[623, 208, 939, 535],
              start_frame=0,
              end_frame=56,
              output_dir=osp.join(empty_frames_dir, '2018-03-07_16-55-00_17-00-00_bus_G331_11_alt'))

prop_18 = edict(video_path=osp.join(video_dir, '2018-03-07_16-55-00_17-00-00_bus_G331.avi'),
              bbox=[694, 128, 984, 418],
              start_frame=0,
              end_frame=532,
              output_dir=osp.join(empty_frames_dir, '2018-03-07_16-55-00_17-00-00_bus_G331_18'))

prop_22 = edict(video_path=osp.join(video_dir,'2018-03-07_16-55-00_17-00-00_bus_G331.avi'),
               bbox=[325, 202, 527, 404],
               start_frame=0,
               end_frame=67,
               output_dir=osp.join(empty_frames_dir, '2018-03-07_16-55-00_17-00-00_bus_G331_22'))

In [25]:
prop_data = prop_11_alt
print(prop_data)
extract_frames(prop_data.video_path, 
               prop_data.bbox, 
               prop_data.start_frame, 
               prop_data.end_frame, 
               prop_data.output_dir)

{'video_path': '/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/videos/2018-03-07_16-55-00_17-00-00_bus_G331.avi', 'bbox': [623, 208, 939, 535], 'start_frame': 0, 'end_frame': 56, 'output_dir': '/mnt/Alfheim/Data/MEVA/umd_cmu_merge_v3/proposals/empty_frames/2018-03-07_16-55-00_17-00-00_bus_G331_11_alt'}
